In [4]:
import zipfile, tarfile
from pathlib import Path
import shutil

ARCHIVE_PATH = Path("dataset100.zip")      
EXTRACT_TO   = Path("UECFOOD100")         
EXTRACT_TO.mkdir(exist_ok=True)

if zipfile.is_zipfile(ARCHIVE_PATH):
    with zipfile.ZipFile(ARCHIVE_PATH, "r") as z:
        z.extractall(EXTRACT_TO)
elif tarfile.is_tarfile(ARCHIVE_PATH):
    with tarfile.open(ARCHIVE_PATH, "r:*") as t:
        t.extractall(EXTRACT_TO)
else:
    raise ValueError(f"Unsupported archive type: {ARCHIVE_PATH.suffix}")

candidates = [p for p in EXTRACT_TO.iterdir() if p.is_dir()]
if len(candidates) == 1:
    BASE_DIR = candidates[0]
else:
    BASE_DIR = EXTRACT_TO

print("Using BASE_DIR:", BASE_DIR)
print("Top‐level contents:", sorted(p.name for p in BASE_DIR.iterdir())[:10])
print("bb_info.txt present? ", (BASE_DIR/"bb_info.txt").exists())
print("category.txt present?", (BASE_DIR/"category.txt").exists())

Using BASE_DIR: UECFOOD100/UECFOOD100
Top‐level contents: ['1', '10', '100', '11', '12', '13', '14', '15', '16', '17']
bb_info.txt present?  False
category.txt present? True


In [6]:
from pathlib import Path

BASE_DIR = Path("UECFOOD100") / "UECFOOD100"
anns = list(BASE_DIR.rglob("bb_info*.txt"))
print("Found annotation files:", anns)

Found annotation files: [PosixPath('UECFOOD100/UECFOOD100/1/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/10/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/100/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/11/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/12/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/13/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/14/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/15/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/16/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/17/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/18/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/19/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/2/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/20/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/21/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/22/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/23/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/24/bb_info.txt'), PosixPath('UECFOOD100/UECFOOD100/25/bb_info.txt'), PosixPa

In [7]:

sample_class = "1"
bb_path = BASE_DIR / sample_class / "bb_info.txt"

print(f"Showing first 10 lines of {bb_path}:\n")
with open(bb_path, "r") as f:
    for i in range(10):
        line = f.readline()
        if not line:
            break
        print(i+1, repr(line.strip()))


Showing first 10 lines of UECFOOD100/UECFOOD100/1/bb_info.txt:

1 'img x1 y1 x2 y2'
2 '1 0 143 370 486'
3 '2 20 208 582 559'
4 '3 2 110 243 410'
5 '4 0 237 286 536'
6 '5 8 28 761 585'
7 '6 0 38 369 310'
8 '7 0 162 383 450'
9 '8 80 31 776 454'
10 '9 2 226 270 470'


In [8]:
!ls UECFOOD100/UECFOOD100/1 | head -n 10

1.jpg
10.jpg
10572.jpg
10586.jpg
10617.jpg
10618.jpg
10621.jpg
10628.jpg
10630.jpg
10633.jpg


In [9]:

from pathlib import Path
from PIL import Image

LABEL_DIR   = Path("yolo_data") / "labels"
LABEL_DIR.mkdir(parents=True, exist_ok=True)

count_files = 0
count_boxes = 0

for class_dir in BASE_DIR.iterdir():
    if not (class_dir.is_dir() and class_dir.name.isdigit()):
        continue
    cls_id  = int(class_dir.name) - 1
    bb_path = class_dir / "bb_info.txt"
    if not bb_path.exists():
        continue

    for line in bb_path.read_text().splitlines():
        parts = line.strip().split()
        if parts[0].lower() == "img":
            continue
        if len(parts) != 5:
            continue

        img_id = parts[0]
        coords = list(map(int, parts[1:]))
        x1, y1, x2, y2 = coords
        img_file = f"{img_id}.jpg"
        img_path = class_dir / img_file
        if not img_path.exists():
            print("Missing image:", img_path)
            continue

        w, h = Image.open(img_path).size

        xc = ((x1 + x2) / 2) / w
        yc = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h

        rel_lbl = Path(class_dir.name) / Path(img_file).with_suffix(".txt")
        out_f   = LABEL_DIR / rel_lbl
        out_f.parent.mkdir(parents=True, exist_ok=True)
        with open(out_f, "w") as f:
            f.write(f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

        count_files += 1
        count_boxes += 1

print("Label files written:", count_files)
print("Total bounding boxes:", count_boxes)

example = next(LABEL_DIR.rglob("*.txt"))
print("Example label file:", example)
print(example.read_text().splitlines()[0])


Label files written: 14611
Total bounding boxes: 14611
Example label file: yolo_data/labels/1/1.txt
0 0.231250 0.524167 0.462500 0.571667


In [10]:
import shutil
from sklearn.model_selection import train_test_split

IMG_DIR = BASE_DIR

all_imgs = list(IMG_DIR.rglob("*.jpg"))
rel_imgs = [p.relative_to(IMG_DIR) for p in all_imgs]

train, val = train_test_split(rel_imgs, test_size=0.2, random_state=42)

print(f"Train ready: {len(train)} images\nVal ready:   {len(val)} images")

YOLO_ROOT = Path("yolo_data")
for split, items in [("train", train), ("val", val)]:
    for rel in items:
        src_img = IMG_DIR / rel
        dst_img = YOLO_ROOT / "images" / split / rel
        dst_img.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(src_img, dst_img)
        src_lbl = YOLO_ROOT / "labels" / rel.with_suffix(".txt")
        if src_lbl.exists():
            dst_lbl = YOLO_ROOT / "labels" / split / rel.with_suffix(".txt")
            dst_lbl.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(src_lbl, dst_lbl)

print("Finished copying train/val images and labels")


Train ready: 11488 images
Val ready:   2873 images
Finished copying train/val images and labels


In [3]:
YOLO_ROOT = Path.cwd() / "yolo_data"
data_yaml = YOLO_ROOT / "data.yaml"
cwd = Path.cwd()

lines = data_yaml.read_text().splitlines()
new_lines = []
for line in lines:
    if line.startswith("train:") or line.startswith("val:"):
        break
    new_lines.append(line)

new_lines.append(f"train: {cwd}/yolo_data/images/train")
new_lines.append(f"val:   {cwd}/yolo_data/images/val")

data_yaml.write_text("\n".join(new_lines) + "\n")
print(data_yaml.read_text())


nc: 100
names:
  - rice
  - eels on rice
  - pilaf
  - chicken-'n'-egg on rice
  - pork cutlet on rice
  - beef curry
  - sushi
  - chicken rice
  - fried rice
  - tempura bowl
  - bibimbap
  - toast
  - croissant
  - roll bread
  - raisin bread
  - chip butty
  - hamburger
  - pizza
  - sandwiches
  - udon noodle
  - tempura udon
  - soba noodle
  - ramen noodle
  - beef noodle
  - tensin noodle
  - fried noodle
  - spaghetti
  - Japanese-style pancake
  - takoyaki
  - gratin
  - sauteed vegetables
  - croquette
  - grilled eggplant
  - sauteed spinach
  - vegetable tempura
  - miso soup
  - potage
  - sausage
  - oden
  - omelet
  - ganmodoki
  - jiaozi
  - stew
  - teriyaki grilled fish
  - fried fish
  - grilled salmon
  - salmon meuniere
  - sashimi
  - grilled pacific saury
  - sukiyaki
  - sweet and sour pork
  - lightly roasted fish
  - steamed egg hotchpotch
  - tempura
  - fried chicken
  - sirloin cutlet
  - nanbanzuke
  - boiled fish
  - seasoned beef with potatoes
  - hamb

In [11]:
!pip install --quiet ultralytics

from ultralytics import YOLO
from pathlib import Path

data_yaml = str(Path.cwd() / "yolo_data" / "data.yaml")

model = YOLO("yolov8n.pt")

results = model.train(
    data=data_yaml,
    epochs=10,        
    imgsz=320,        
    batch=4,          
    device=0,         
    name="uecfood100_test",
    augment=False,    
    cache=False     
)

print("Quick test run finished. Results:\n", results)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires gast<=0.4.0,>=0.2.1, but you have gast 0.5.2 which is incompatible.
accelerate 1.6.0 requires torch>=2.0.0, but you have torch 1.13.1 which is incompatible.
New https://pypi.org/project/ultralytics/8.3.129 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.127 🚀 Python-3.9.6 torch-1.13.1 CUDA:0 (Tesla V100-PCIE-32GB, 32494MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/user/saisriko/yolo_data/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False,

train: Scanning /user/saisriko/yolo_data/labels/train/1... 11488 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11488/11488 [00:06<00:00, 1762.13it/s]

train: /user/saisriko/yolo_data/images/train/1/85.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/1/87.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/1/98.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/16/1503.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/16/1581.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/16/1585.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/30/2902.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/35/3394.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/36/82.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/36/84.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yolo_data/images/train/40/83.jpg: corrupt JPEG restored and saved
train: /user/saisriko/yol

train: New cache created: /user/saisriko/yolo_data/labels/train/1.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 25.4±10.4 MB/s, size: 22.9 KB)


val: Scanning /user/saisriko/yolo_data/labels/val/1... 2873 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2873/2873 [00:01<00:00, 1810.25it/s]

...

                   all       2873       2873      0.269      0.422      0.315       0.24

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10     0.529G     0.8236      2.607      1.191          4        320: 100%|██████████| 2872/2872 [04:56<00:00,  9.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 360/360 [00:16<00:00, 22.12it/s]


                   all       2873       2873      0.266       0.45      0.326      0.247

10 epochs completed in 0.870 hours.
Optimizer stripped from runs/detect/uecfood100_test2/weights/last.pt, 6.9MB
Optimizer stripped from runs/detect/uecfood100_test2/weights/best.pt, 6.9MB

Validating runs/detect/uecfood100_test2/weights/best.pt...
Ultralytics 8.3.127 🚀 Python-3.9.6 torch-1.13.1 CUDA:0 (Tesla V100-PCIE-32GB, 32494MiB)
Model summary (fused): 72 layers, 3,340,724 parameters, 0 gradients, 9.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 360/360 [00:13<00:00, 26.43it/s]


                   all       2873       2873      0.264      0.451      0.326      0.247
                  rice        131        131      0.358      0.573      0.445      0.321
          eels on rice         25         25      0.366          1      0.959      0.803
                 pilaf         23         23      0.292      0.478      0.353      0.196
chicken-'n'-egg on rice         25         25      0.129       0.24      0.145      0.132
   pork cutlet on rice         36         36      0.478      0.472       0.42      0.338
            beef curry         50         50       0.32       0.64      0.469      0.384
                 sushi         36         36       0.37      0.474      0.414      0.309
          chicken rice         22         22       0.18        0.2      0.187      0.141
            fried rice         28         28      0.261      0.714      0.382      0.296
          tempura bowl         30         30      0.427      0.497      0.466      0.341
              bibimb

In [12]:
metrics = model.val(
    data=data_yaml,
    imgsz=320,
    batch=4,
    device=0
)
print("YOLOv8 validation metrics:\n", metrics)


Ultralytics 8.3.127 🚀 Python-3.9.6 torch-1.13.1 CUDA:0 (Tesla V100-PCIE-32GB, 32494MiB)
Model summary (fused): 72 layers, 3,340,724 parameters, 0 gradients, 9.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 291.4±167.5 MB/s, size: 91.2 KB)


val: Scanning /user/saisriko/yolo_data/labels/val/1.cache... 2873 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2873/2873 [00:00<?, ?it/s]

val: /user/saisriko/yolo_data/images/val/1/81.jpg: corrupt JPEG restored and saved
val: /user/saisriko/yolo_data/images/val/35/3472.jpg: corrupt JPEG restored and saved
val: /user/saisriko/yolo_data/images/val/36/87.jpg: corrupt JPEG restored and saved
val: /user/saisriko/yolo_data/images/val/36/99.jpg: corrupt JPEG restored and saved
val: /user/saisriko/yolo_data/images/val/67/86.jpg: corrupt JPEG restored and saved
val: /user/saisriko/yolo_data/images/val/99/86.jpg: corrupt JPEG restored and saved
val: /user/saisriko/yolo_data/images/val/99/99.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 719/719 [00:16<00:00, 43.03it/s]


                   all       2873       2873      0.263      0.452      0.327      0.248
                  rice        131        131       0.36      0.579      0.446      0.321
          eels on rice         25         25      0.365          1      0.959      0.814
                 pilaf         23         23      0.291      0.478      0.353      0.196
chicken-'n'-egg on rice         25         25      0.126       0.24      0.145      0.132
   pork cutlet on rice         36         36      0.466      0.472      0.421      0.339
            beef curry         50         50      0.318       0.64      0.469      0.384
                 sushi         36         36       0.37      0.474      0.414      0.311
          chicken rice         22         22      0.188      0.211      0.178      0.134
            fried rice         28         28      0.266      0.714      0.382      0.296
          tempura bowl         30         30      0.429        0.5      0.486      0.359
              bibimb